<a href="https://colab.research.google.com/github/steffenmodest/Python_Examples/blob/main/M_Net_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

rs = 23  # fester Random Seed

Load Data

In [2]:
from tensorflow.keras.datasets import fashion_mnist

# Fashion MNIST Daten laden, wir wollen nur die Trainingsdaten und verwerfen die Testdaten
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

4423680/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


Preprocess data

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, FunctionTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Plätte 2D Bild in einen Vektor:
# Reshape behält die Anzahl Element in der ersten Dimension (len(X_orig) -> #Bilder)
# die restlichen Zellen des Arrays (-1 -> Pixel) bekommen alle ihre eigene Dimension
X = X_train.reshape(len(X_train), -1).astype('float64')

# Dimensionen um den Mittelpunkt zentrieren
preproccessing = StandardScaler()
X = preproccessing.fit_transform(X)

print ("Originaldaten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_train.shape, np.mean(X_train), np.std(X_train)))

print ("Vorbereitete Daten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X.shape, np.mean(X), np.std(X)))

Originaldaten:
Shape: (60000, 28, 28), Mean: 72.940352, STD: 90.021182
Vorbereitete Daten:
Shape: (60000, 784), Mean: -0.000000, STD: 1.000000


Create Hidden Activations

In [4]:
H = np.random.rand(60000, 40)
print(H.shape)
print(H)

(60000, 40)
[[0.50144111 0.25627317 0.75818555 ... 0.83165346 0.32339153 0.80343182]
 [0.24236432 0.42358975 0.96474852 ... 0.39492283 0.55355526 0.27501597]
 [0.83081876 0.38320399 0.40628034 ... 0.07433797 0.38638323 0.28643729]
 ...
 [0.89013284 0.70698866 0.8855743  ... 0.03022112 0.90898571 0.70650297]
 [0.06004864 0.32688667 0.02347121 ... 0.22818898 0.16362    0.47457009]
 [0.6334994  0.8709885  0.10537503 ... 0.70141479 0.13736858 0.840402  ]]


1st Forward Learn

In [5]:
import numpy as np
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(H, X)
print(reg.score(H, X))
X_pred = reg.predict(H)
print(f'MAE {mean_absolute_error(X_pred, X)}')
print(f'MSE {mean_squared_error(X_pred, X)}')
print(f'R2 {r2_score(X_pred, X)}')
print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

W = reg.coef_
print(W.shape)
WI = reg.intercept_
print(WI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.000617807112771911
MAE 0.7514320032090278
MSE 0.9993821928872308
R2 -1690.6970744293487
R2 vw -1616.628175884802
R2 ua -1690.6970744293487
(784, 40)
(784,)


1st Back Activations

In [6]:
XT = X.transpose()
print(f'shape of X_train {X.shape} and X_train transposed {XT.shape}')

reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 {r2_score(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_new = reg.coef_
print(H_new.shape)
HI = reg.intercept_
print(HI.shape)

shape of X_train (60000, 784) and X_train transposed (784, 60000)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.5588333663432447
MAE 0.4126441031398016
MSE 0.37076434710013095
R2 0.1736427065058122
R2 vw 0.21055781521502243
R2 ua 0.1736427065058122
(60000, 40)
(60000,)


2nd Forward learn - result are Coherent Weights

In [9]:
reg = LinearRegression().fit(H_new, X)
print(reg.score(H_new, X))
X_pred = reg.predict(H_new)
print(f'MAE {mean_absolute_error(X_pred, X)}')
print(f'MSE {mean_squared_error(X_pred, X)}')
print(f'R2 {r2_score(X_pred, X)}')
print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

W = reg.coef_
print(W.shape)
WI = reg.intercept_
print(WI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.729139582586839
MAE 0.33424101117884764
MSE 0.2708604174131617
R2 0.5766778683431001
R2 vw 0.6285204864997156
R2 ua 0.5766778683431001
(784, 40)
(784,)


2nd Back Activation - final representation of X_train in Hidden Activations

In [8]:
reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_new = reg.coef_
print(H_new.shape)
HI = reg.intercept_
print(HI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.7150637909382017
MAE 0.3058959510569444
MSE 0.23946549774700157
R2 vw 0.6015233708199004
R2 ua 0.5748435957764805
(60000, 40)
(60000,)


KNN test with Hidden Activations

In [12]:
# X = [[0], [1], [2], [3]]
# y = [0, 0, 1, 1]

# Plätte 2D Bild in einen Vektor:
# Reshape behält die Anzahl Element in der ersten Dimension (len(X_orig) -> #Bilder)
# die restlichen Zellen des Arrays (-1 -> Pixel) bekommen alle ihre eigene Dimension
X_p_test = X_test.reshape(len(X_test), -1).astype('float64')

# Dimensionen um den Mittelpunkt zentrieren
preproccessing = StandardScaler()
X_p_test = preproccessing.fit_transform(X_p_test)

print ("Originaldaten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_test.shape, np.mean(X_test), np.std(X_test)))

print ("Vorbereitete Daten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_p_test.shape, np.mean(X_p_test), np.std(X_p_test)))

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(H_new, y_train)

# print(neigh.predict([[1.1]]))

# print(neigh.predict_proba([[0.9]]))

Originaldaten:
Shape: (10000, 28, 28), Mean: 73.146567, STD: 89.873259
Vorbereitete Daten:
Shape: (10000, 784), Mean: -0.000000, STD: 1.000000


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

Back Activation of Test Data

In [21]:
XT = X_p_test.transpose()
print(f'shape of X_p_train {X_p_test.shape} and X_p_train transposed {XT.shape}')

reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 {r2_score(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_test = reg.coef_
print(H_test.shape)
HI_test = reg.intercept_
print(HI_test.shape)

shape of X_p_train (10000, 784) and X_p_train transposed (784, 10000)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.7287418892074771
MAE 0.2947673443153052
MSE 0.2279441717298359
R2 0.5967567589822452
R2 vw 0.6277720345024742
R2 ua 0.5967567589822452
(10000, 40)
(10000,)


In [22]:
# Predicting the Test set results
y_pred = neigh.predict(H_test)

In [25]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[830   1  17  25   5   0 118   0   4   0]
 [  8 967   0  15   4   0   5   0   1   0]
 [ 30   2 790  11  84   1  77   0   5   0]
 [ 44  12  12 853  45   0  31   0   3   0]
 [  8   1 122  36 756   0  75   0   2   0]
 [  1   0   1   1   0 882   0  75   2  38]
 [173   4 111  31  77   1 595   0   8   0]
 [  0   0   0   0   0  23   0 934   0  43]
 [  9   1   4   7   8  14   9   3 942   3]
 [  1   0   0   0   0   7   0  48   0 944]]


In [27]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8493
              precision    recall  f1-score   support

           0       0.75      0.83      0.79      1000
           1       0.98      0.97      0.97      1000
           2       0.75      0.79      0.77      1000
           3       0.87      0.85      0.86      1000
           4       0.77      0.76      0.76      1000
           5       0.95      0.88      0.91      1000
           6       0.65      0.59      0.62      1000
           7       0.88      0.93      0.91      1000
           8       0.97      0.94      0.96      1000
           9       0.92      0.94      0.93      1000

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000

